In [1]:
%load_ext autoreload
%autoreload 1

In [2]:
import polars as pl
import numpy as np
import random

In [3]:
df_t = pl.read_csv('~/CIS_4496_Project/data/raw/transactions_train.csv')
df_t.head()

t_dat,customer_id,article_id,price,sales_channel_id
str,str,i64,f64,i64
"""2018-09-20""","""000058a12d5b43...",663713001,0.050831,2
"""2018-09-20""","""000058a12d5b43...",541518023,0.030492,2
"""2018-09-20""","""00007d2de82675...",505221004,0.015237,2
"""2018-09-20""","""00007d2de82675...",685687003,0.016932,2
"""2018-09-20""","""00007d2de82675...",685687004,0.016932,2


In [4]:
df_t.height

31788324

In [5]:
from scripts.df_ct_combine import *
df_t_u = combine()
df_t_u

customer_id,articles
str,list[str]
"""000058a12d5b43...","[""663713001"", ""541518023"", ... ""794321007""]"
"""00007d2de82675...","[""505221004"", ""685687003"", ... ""808651003""]"
"""00083cda041544...","[""688873012"", ""501323011"", ... ""783504004""]"
"""0008968c0d451d...","[""531310002"", ""529841001"", ... ""893059004""]"
"""000aa7f0dc06cd...","[""501820043"", ""501820043"", ... ""903309001""]"
"""001127bffdda10...","[""397068015"", ""714415001"", ... ""717490010""]"
"""001ea4e9c54f7e...","[""652075001"", ""670295001"", ... ""863646005""]"
"""001fd23db1109a...","[""631744002"", ""562252035"", ... ""828536004""]"
"""0021da829b898f...","[""649356002"", ""579941002"", ... ""625229004""]"


In [6]:
# df_t_u.to_pandas().to_csv('df_t_u.csv',index=False)

In [7]:
from sklearn.model_selection import train_test_split

# customer_train, customer_test = train_test_split(df_t_u, test_size = 0.2, random_state=9)

In [8]:
t_train, t_test = train_test_split(df_t_u['articles'][0], test_size=0.2, random_state=9)
t_train_train = pl.Series('t_train_train', [t_train])
t_train_test = pl.Series('t_train_test', [t_test])

In [9]:
for to_split in df_t_u['articles'][1:]:
    
    if len(to_split) > 1:
        t_train, t_test = train_test_split(to_split, test_size=0.2, random_state=9)

        t_train_train = t_train_train.append(pl.Series([t_train]))
        t_train_test = t_train_test.append(pl.Series([t_test]))
    else:
        t_train_train = t_train_train.append(pl.Series([to_split]))
        t_train_test = t_train_test.append(pl.Series([None], dtype=pl.List(inner=str)))

In [15]:
df_tts = df_t_u.hstack([t_train_train]).hstack([t_train_test]).rename({"t_train_train": "train", "t_train_test": "test"})
df_tts

customer_id,articles,train,test
str,list[str],list[str],list[str]
"""000058a12d5b43...","[""663713001"", ""541518023"", ... ""794321007""]","[""727808001"", ""750424014"", ... ""541518023""]","[""852643001"", ""723529001"", ... ""351484002""]"
"""00007d2de82675...","[""505221004"", ""685687003"", ... ""808651003""]","[""522992001"", ""664075001"", ... ""708379003""]","[""787147002"", ""685687001"", ... ""515815001""]"
"""00083cda041544...","[""688873012"", ""501323011"", ... ""783504004""]","[""695263003"", ""723469007"", ... ""683001007""]","[""568571002"", ""687033001"", ... ""559630003""]"
"""0008968c0d451d...","[""531310002"", ""529841001"", ... ""893059004""]","[""705732001"", ""351933029"", ... ""737236003""]","[""736530014"", ""685044001"", ... ""856440005""]"
"""000aa7f0dc06cd...","[""501820043"", ""501820043"", ... ""903309001""]","[""631848002"", ""758064001"", ... ""700819006""]","[""631848002"", ""631848002"", ... ""758049001""]"
"""001127bffdda10...","[""397068015"", ""714415001"", ... ""717490010""]","[""781758045"", ""607154015"", ... ""849648003""]","[""787946002"", ""763280003"", ... ""742912002""]"
"""001ea4e9c54f7e...","[""652075001"", ""670295001"", ... ""863646005""]","[""860885001"", ""731407005"", ... ""788575003""]","[""851340001"", ""760111006"", ... ""669708001""]"
"""001fd23db1109a...","[""631744002"", ""562252035"", ... ""828536004""]","[""841367001"", ""795621001"", ... ""733098018""]","[""578476001"", ""408649007"", ... ""800807001""]"
"""0021da829b898f...","[""649356002"", ""579941002"", ... ""625229004""]","[""625229004"", ""649356002"", ""629760002""]","[""579941002""]"


In [16]:
df_tts.to_pandas().to_csv('~/data/train_test_split/df_tts.csv',index=False)